In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
import warnings
warnings.filterwarnings(action = 'ignore')

In [2]:
#위도 =111.195km , 경도 88.8km , 2km 박스일 때 반경  1.414
def dis_to_lat(km):
    return km/111.195
def dis_to_lon(km):
    return km/88.8

def lat_to_dis(lat):
    return lat*111.195
def lon_to_dis(lon):
    return lon*88.8

def get_xy(data,lon, lat,km):
    lat_size = dis_to_lat(km)
    lon_size = dis_to_lon(km)
    return data[(data['위도']<=lat+lat_size/2) & (data['위도']>=lat-lat_size/2) &
                 (data['경도']<=lon+lon_size/2) & (data['경도']>=lon-lon_size/2)]

In [3]:
def get_traffic_light(lon, lat, km):
    global df,df2
    df = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울시 신호등 관련 정보.csv",encoding='CP949')
    df['X좌표'] = df['X좌표'].astype(float)
    df['Y좌표'] = df['Y좌표'].astype(float)
    df['geometry'] = df.apply(lambda row : Point([row['X좌표'], row['Y좌표']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry')
    df.crs = from_string("+proj=tmerc +lat_0=38 +lon_0=127 +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +units=m +no_defs")
    wgs84 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    df = df.to_crs(wgs84)
    df['lon'] = df['geometry'].map(lambda t : t.x)
    df['lat'] = df['geometry'].map(lambda t : t.y)
    df = df.rename({'lat':'위도','lon':'경도'},axis='columns')
    df = df[['신호등수량','위도','경도']]
    return get_xy(df, lon, lat, km)

In [4]:
def get_crosswalk(lon,lat,km) :
    global df3
    df3 = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울시 횡단보도 위치정보 (좌표계_ WGS1984).csv",encoding='CP949')
    df3 = df3.drop(['순번', '상태', '횡단보도종류코드', '가로길이', '세로길이', '화살표시수량', '화살표시길이',
       '고가', '구경찰서코드', '구코드', '동코드', '지번', '신경찰서코드', '작업구분', '표출구분', '도로구분',
       '관할사업소', '신규정규화ID', '설치일', '교체일', '이력ID', '공사관리번호', '횡단보도관리번호.1',
       '공사형태'], axis='columns')
    return get_xy(df3, lon, lat, km)

In [5]:
def get_bus_stop(lon,lat,km) :
    global df4
    df4 = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울특별시 버스정류소 위치정보.csv",encoding='CP949')
    df4 = df4.drop(['정류소번호'],axis=1)
    df4 = df4.rename({'X좌표':'경도','Y좌표':'위도'},axis=1)
    df4 = df4[['정류소명','위도','경도']]
    
    return get_xy(df4,lon,lat,km)

In [6]:
def get_subway(lon,lat,km) :
    global df5
    df5 = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/station_coordinate.csv")
    df5 = df5.drop(['line','code'],axis=1)
    df5 = df5.rename({'lat':'위도','lng':'경도'},axis=1)
    return get_xy(df5,lon,lat,km)

In [7]:
def mean_top10(environment):
    eev_sum = 0
    for i in range(0,9):
        eev = environment[i]
        eev_sum += len(environment[i])
        
    mean_ev = eev_sum / 10
    return mean_ev

In [11]:
def score_ev(mean_tl, mean_cw, mean_bs, mean_sb,target_tl,target_cw,target_bs,target_sb):
    
    if(mean_tl != 0):
        score_tl = (len(target_tl)/mean_tl)
        if (score_tl > 1):
            score_tl = 1
    else:
        score_tl = 0

    if(mean_cw != 0):
        score_cw = (len(target_cw)/mean_cw)
        if (score_cw > 1):
            score_cw = 1
    else:
        score_cw = 0
    
    if(mean_bs != 0):
        score_bs = (len(target_bs)/mean_bs)
        if (score_bs > 1):
            score_bs = 1
    else:
        score_bs = 0

    if (mean_sb != 0): 
        score_sb = (len(target_sb)/mean_sb)
        if (score_sb > 1):
            score_sb = 1
    else:
        score_sb = 1
        
    traffic_score = score_tl + score_cw + score_bs + score_sb
    
    traffic_score = traffic_score * 25
    
    return traffic_score

In [29]:
def get_score(top10_df,target_lon,target_lat,km):
    tl=[]
    cw=[]
    bs=[]
    sb = []
    
    for tup in zip(list(top10_df['경도']),list(top10_df['위도'])):
        tl.append(get_traffic_light(tup[0], tup[1], 1))
        cw.append(get_crosswalk(tup[0], tup[1], 1))
        bs.append(get_bus_stop(tup[0], tup[1], 1))
        sb.append(get_subway(tup[0], tup[1], 1))
        
    mean_tl = mean_top10(tl)
    mean_cw = mean_top10(cw)
    mean_bs = mean_top10(bs)
    mean_sb = mean_top10(sb)
    
    target_tl = get_traffic_light(target_lon,target_lat,km)
    target_cw = get_crosswalk(target_lon,target_lat,km)
    target_bs = get_bus_stop(target_lon,target_lat,km)
    target_sb = get_subway(target_lon,target_lat,km)
    
    traffic_score = score_ev(mean_tl,mean_cw,mean_bs,mean_sb,target_tl,target_cw,target_bs,target_sb)

    return traffic_score    

In [30]:
#126.955198,37.602663
#test_df = df4.iloc[:10]
#get_score(test_df,126.955198,37.602663,0.5)
get_crosswalk(126.955198,37.602663,0.6)
get_bus_stop(126.955198,37.602663,0.5)
get_subway(126.955198,37.602663,4)
print(80)

80


In [33]:
test = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/냉면집.csv",encoding = 'CP949')
test

,이름,주소,위도,경도
0,진미평양냉면,서울시 강남구 학동로 305-3 정각빌딩 1층,37.516145,127.036042
1,서관면옥,서울시 서초구 서초대로 56길 11 1층,37.493991,127.017269
2,오장동흥남집 본점,서울시 중구 마른내로 114,37.564341,127.000542
3,우래옥 본점,서울시 중구 창경궁로 62-29,37.568219,126.998714
4,해주냉면,서울시 송파구 백제고분로7길 8-16 1층,37.509653,127.079773
5,을밀대 본점,서울시 마포구 숭문길 24,37.547493,126.945532
6,오장동 함흥냉면,서울시 중구 마른내로 108,37.564272,126.999884
7,정인면옥,서울시 영등포구 국회대로76길 10 1층,37.530656,126.921558
8,남포면옥,서울시 중구 을지로3길 24,37.567146,126.981676
9,평양면옥,서울시 중구 장충단로 207,37.563030,127.006470


In [40]:
score = get_score(test,127.036042,37.516145,0.5)


score += 60
score

82.93002014495042

In [42]:
target_cw = get_crosswalk(127.036042,37.516145,0.5)
len(target_cw)

46

In [44]:
target_bs = get_bus_stop(127.036042,37.516145,0.5)
target_bs

,정류소명,위도,경도
9588,세관앞,37.515682,127.036104
9589,서울세관,37.517034,127.035466
9590,서울세관,37.517327,127.035215
9593,세관앞,37.515655,127.036434
9599,임페리얼팰리스호텔.워터게이트빌딩,37.514336,127.035430
9606,서울세관사거리,37.515297,127.034811


In [47]:
target_sw = get_subway(127.036042,37.516145,1)
target_sw

,name,위도,경도
334,강남구청,37.517186,127.041280
335,학동,37.514229,127.031656
566,강남구청,37.517186,127.041280
